# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Reading CSV city weather

city_weather = pd.read_csv("city_weather.csv")
city_weather.dropna()
city_weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Jiexiu,0,CN,1584581790,11,37.02,111.91,50.63,10.00
1,Lakatoro,75,VU,1584581644,79,-16.10,167.42,84.20,6.93
2,Vastseliina,16,EE,1584581871,75,57.73,27.29,42.80,13.42
3,Hofn,100,IS,1584581871,74,64.25,-15.21,33.71,22.84
4,Vaini,75,TO,1584581872,69,-21.20,-175.20,82.40,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

gmaps.configure(api_key=g_key)

# reduce df to lang and lat

locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"].astype(float)


In [4]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_weather["Humidity"]),
                                 point_radius = 3)

# Display figure
fig = gmaps.figure(center=(0,0), zoom_level=2)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness

# create happy places as pre comfortable weather condition

happy_df = city_weather.loc[(city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & (city_weather["Wind Speed"] < 10) & (city_weather["Cloudiness"] == 0), :]
happy_df = happy_df.dropna(how='any')
happy_df.reset_index(inplace=True)
del happy_df['index']
happy_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Lázaro Cárdenas,0,MX,1584581879,80,17.96,-102.20,74.39,2.80
1,Jesús Carranza,0,MX,1584581882,94,17.43,-95.03,72.05,5.32
2,São João da Barra,0,BR,1584581887,88,-21.64,-41.05,76.01,3.67
3,Porbandar,0,IN,1584581909,72,21.64,69.61,77.09,9.13
4,Christchurch,0,NZ,1584581744,27,-43.53,172.63,77.00,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df = happy_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
# set up parameters to search hotels in 50000 radious
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    
    lat = row["Lat"]
    lng = row["Lng"]
    
 
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request to fetch data
    hotels = requests.get(base_url, params=params)

    hotel_json = hotels.json()

    
    # exception handeling
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Lázaro Cárdenas,0,MX,1584581879,80,17.96,-102.20,74.39,2.80,Hotel de Casablanca
1,Jesús Carranza,0,MX,1584581882,94,17.43,-95.03,72.05,5.32,Real Inn Nuevo Morelos
2,São João da Barra,0,BR,1584581887,88,-21.64,-41.05,76.01,3.67,Ramada by Wyndham Campos dos Goytacazes Hotel ...
3,Porbandar,0,IN,1584581909,72,21.64,69.61,77.09,9.13,The Fern Leo Beach Resort
4,Christchurch,0,NZ,1584581744,27,-43.53,172.63,77.00,4.70,"Chateau on the Park - Christchurch, a DoubleTr..."
5,Becerril,0,CO,1584581925,51,9.70,-73.28,79.79,3.78,Bapa Hotel Colonial
6,Sur,0,OM,1584581928,39,22.57,59.53,72.77,6.04,Sama Ras Al Jinz
7,Bilma,0,NE,1584581930,11,18.69,12.92,70.07,7.11,
8,San Juan,0,AR,1584581935,29,-31.54,-68.54,74.57,7.38,Del Bono Park Hotel Spa & Casino
9,Ciudad Guayana,0,VE,1584581948,73,8.35,-62.65,76.55,7.90,Hotel Hosteria Waipá


In [7]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df=hotel_df.dropna(how='any')
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Lázaro Cárdenas,0,MX,1584581879,80,17.96,-102.20,74.39,2.80,Hotel de Casablanca
1,Jesús Carranza,0,MX,1584581882,94,17.43,-95.03,72.05,5.32,Real Inn Nuevo Morelos
2,São João da Barra,0,BR,1584581887,88,-21.64,-41.05,76.01,3.67,Ramada by Wyndham Campos dos Goytacazes Hotel ...
3,Porbandar,0,IN,1584581909,72,21.64,69.61,77.09,9.13,The Fern Leo Beach Resort
4,Christchurch,0,NZ,1584581744,27,-43.53,172.63,77.00,4.70,"Chateau on the Park - Christchurch, a DoubleTr..."
5,Becerril,0,CO,1584581925,51,9.70,-73.28,79.79,3.78,Bapa Hotel Colonial
6,Sur,0,OM,1584581928,39,22.57,59.53,72.77,6.04,Sama Ras Al Jinz
8,San Juan,0,AR,1584581935,29,-31.54,-68.54,74.57,7.38,Del Bono Park Hotel Spa & Casino
9,Ciudad Guayana,0,VE,1584581948,73,8.35,-62.65,76.55,7.90,Hotel Hosteria Waipá
10,Nichlaul,0,IN,1584581970,45,27.32,83.73,73.31,0.58,Barahi Jungle Lodge


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content=[f"Hotel loc: {hotel}" for hotel in hotel_info])
                           
fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))